In [2]:
# DS 440 Data Exploration

# Spring 2023
### Instructor: Prof. John Yen
### Group Name: Philly Boys
### Team Members: Andrew cavnaugh, Aman Arora


In [3]:
import pyspark
import csv

In [4]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, LongType
from pyspark.sql.functions import col, column
from pyspark.sql.functions import expr
from pyspark.sql.functions import split
from pyspark.sql.functions import array_contains
from pyspark.sql import Row
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, IndexToString
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.feature import PCA

In [5]:
#Local Mode
#ss = SparkSession.builder.master("local").appName("Data Analysis Cusltering").getOrCreate()
#Cluster Mode
ss = SparkSession.builder.appName("Data Exploration Analysis Clustering").getOrCreate()

In [6]:
## Read in Data

In [7]:
Scanners_df = ss.read.csv("/storage/home/aka5922/data/complete_Day1_2021profile.csv", header= True, inferSchema=True )
Scanners2_df = ss.read.csv("/storage/home/aka5922/data/complete_Day2_2021profile.csv", header= True, inferSchema=True )

In [8]:
#Filter out extreme scanners

In [9]:
multi_port_scanners= Scanners_df.where(col("numports")>1)
multi_port_scanners_d2 = Scanners2_df.where(col("numports")>1)

In [10]:
multi_port_scanners_count = multi_port_scanners.count()
multi_port_scanners_count_d2 = multi_port_scanners_d2.count()

In [11]:
ScannersCount_byNumPorts = multi_port_scanners.groupby("numports").count()
ScannersCount_byNumPorts_d2 = multi_port_scanners_d2.groupby("numports").count()

In [12]:
SortedNumPorts_DF= ScannersCount_byNumPorts.orderBy("numports", ascending = False)
SortedNumPorts2_DF = ScannersCount_byNumPorts_d2.orderBy("numports", ascending = False)

In [13]:
max_non_rare_NumPorts= 1000

In [14]:
extreme_scanners = Scanners_df.where(col("numports") > max_non_rare_NumPorts)
extreme_scanners_d2 = Scanners2_df.where(col("numports") > max_non_rare_NumPorts)

In [15]:
#path2="/storage/home/aka5922/Extreme_Scanners.csv"
#path2_2="/storage/home/aka5922/Extreme_Scanners_D2.csv"
#extreme_scanners.write.option("header",True).csv(path2)
#extreme_scanners_day2.write.option("header",True).csv(path2_2)

In [16]:
non_extreme_multi_port_scanners = Scanners_df.where(col("numports") <= max_non_rare_NumPorts).where(col("numports") > 1)
non_extreme_multi_port_scanners_d2 = Scanners2_df.where(col("numports") <= max_non_rare_NumPorts).where(col("numports") > 1)


In [17]:
# (a)
NEMP_Scanners_df=non_extreme_multi_port_scanners.withColumn("Ports_Array", split(col("ports_scanned_str"), "-") )
NEMP_Scannersd2_df=non_extreme_multi_port_scanners_d2.withColumn("Ports_Array_2", split(col("ports_scanned_str"), "-") )

In [ ]:
## Ports Scanned

In [15]:
Ports_Scanned_RDD = NEMP_Scanners_df.select("Ports_Array").rdd
Ports_Scanned_RDD_d2 = NEMP_Scannersd2_df.select("Ports_Array_2").rdd

In [16]:
Ports_list_RDD = Ports_Scanned_RDD.map(lambda row: row[0] )
Ports_list_RDD_d2 = Ports_Scanned_RDD_d2.map(lambda row: row[0] )

In [17]:
Ports_list2_RDD = Ports_list_RDD.flatMap(lambda x: x )
Ports_list2_RDD_d2 = Ports_list_RDD_d2.flatMap(lambda x: x )


In [18]:
Port_count_RDD = Ports_list2_RDD.map(lambda x: (x, 1))
Port_count_RDD_d2 = Ports_list2_RDD_d2.map(lambda x: (x, 1))


In [19]:
Port_count_total_RDD = Port_count_RDD.reduceByKey(lambda x,y: x+y)
Port_count_total_RDD_d2 = Port_count_RDD_d2.reduceByKey(lambda x,y: x+y)


In [20]:
Sorted_Count_Port_RDD = Port_count_total_RDD.map(lambda x: (x[1], x[0])).sortByKey( ascending = False)
Sorted_Count_Port_RDD_d2 = Port_count_total_RDD_d2.map(lambda x: (x[1], x[0])).sortByKey( ascending = False)

In [21]:
top_k_ports = 200
Sorted_Count_Port_RDD.take(top_k_ports)
Sorted_Count_Port_RDD_day2.take(top_k_ports)


[(256111, '17128'),
 (256071, '17138'),
 (255938, '17132'),
 (255841, '17136'),
 (255675, '17130'),
 (255267, '17140'),
 (187424, '17134'),
 (187143, '17142'),
 (115588, '8080'),
 (115503, '80'),
 (105589, '23'),
 (38254, '62904'),
 (37998, '81'),
 (32353, '34220'),
 (32325, '34226'),
 (32309, '34224'),
 (32293, '34216'),
 (32287, '34218'),
 (32231, '33970'),
 (32098, '33968'),
 (32036, '33960'),
 (32025, '33972'),
 (31993, '33964'),
 (31974, '34228'),
 (31884, '33962'),
 (31417, '8443'),
 (31349, '49152'),
 (30442, '7574'),
 (28626, '52475'),
 (26667, '5555'),
 (26164, '1023'),
 (26016, '2323'),
 (25187, '52869'),
 (24959, '37215'),
 (21922, '8081'),
 (19088, '34222'),
 (19052, '33974'),
 (19045, '34230'),
 (18669, '33966'),
 (17010, '8181'),
 (16917, '60001'),
 (9490, '8550'),
 (8476, '445'),
 (6870, '0'),
 (6401, '30740'),
 (5482, '22'),
 (4913, '17018'),
 (4900, '54046'),
 (4708, '43293'),
 (4307, '13599'),
 (4148, '57628'),
 (4127, '5407'),
 (4110, '64030'),
 (4084, '64284'),
 (40

In [22]:
Sorted_Ports_RDD= Sorted_Count_Port_RDD.map(lambda x: x[1] )
Sorted_Ports_RDD_d2= Sorted_Count_Port_RDD_d2.map(lambda x: x[1] )

Top_Ports_list = Sorted_Ports_RDD.take(top_k_ports)
Top_Ports_list_d2 = Sorted_Ports_RDD_d2.take(top_k_ports)

In [23]:
Top_Ports_combine = Top_Ports_list
for i in range(0,199):
    same_port_found = False
    for j in range(0,199):
        if Top_Ports_combine[j] == Top_Ports_list_d2[i]:
            #print('same port'+Top_Ports_list_day2[i])
            same_port_found = True
    if same_port_found == False:
        Top_Ports_combine.append(Top_Ports_list_d2[i])


In [24]:
from pyspark.sql.functions import array_contains

In [ ]:
top_k_ports_combine = len(Top_Ports_combine)
print(top_k_ports_combine)

In [26]:
NEMP_Scanners2_both_df = NEMP_Scanners_df
NEMP_Scanners2_d2_both_df = NEMP_Scannersd2_df

for j in range(0, top_k_ports_combine):
    # "Port" + Top_Ports_list[i]  is the name of each new feature created through One Hot Encoding Top_Ports_list
    NEMP_Scanners3_both_df = NEMP_Scanners2_both_df.withColumn("Port" + Top_Ports_combine[j], array_contains("Ports_Array", Top_Ports_combine[j]))
    NEMP_Scanners2_both_df = NEMP_Scanners3_both_df
for i in range(0, top_k_ports_combine):
    # "Port" + Top_Ports_list[i]  is the name of each new feature created through One Hot Encoding Top_Ports_list
    NEMP_Scanners3_day2_both_df = NEMP_Scanners2_d2_both_df.withColumn("Port" + Top_Ports_combine[i], array_contains("Ports_Array_2", Top_Ports_combine[i]))
    NEMP_Scanners2_day2_both_df = NEMP_Scanners3_d2_both_df


In [27]:
input_features = [ ]
for i in range(0, top_k_ports_combine):
    input_features.append( "Port"+ Top_Ports_combine[i] )

In [ ]:
#K means clustering

In [28]:
va = VectorAssembler().setInputCols(input_features).setOutputCol("features")

In [29]:
data_day1= va.transform(NEMP_Scanners2_both_df)
data_day2= va.transform(NEMP_Scanners2_d2_both_df)

In [31]:
total_clusters = top_k_ports_both
km = KMeans(featuresCol= "features", predictionCol="prediction").setK(top_k_ports_both).setSeed(123)
#km.explainParams()

In [32]:
kmModel_day1=km.fit(data_day1)
kmModel_day2=km.fit(data_day2)

In [33]:
predictions_day1 = kmModel_day1.transform(data_day1)
predictions_day2 = kmModel_day2.transform(data_day2)

In [34]:
Cluster1_day1_df=predictions_day1.where(col("prediction")==0)
Cluster1_day2_df=predictions_day2.where(col("prediction")==0)

In [35]:
summary_day1 = kmModel_day1.summary
summary_day2 = kmModel_day2.summary

In [36]:
evaluator = ClusteringEvaluator()
silhouette_day1 = evaluator.evaluate(predictions_day1)
silhouette_day2 = evaluator.evaluate(predictions_day2)

In [37]:
centers_day1 = kmModel_day1.clusterCenters()
centers_day2 = kmModel_day2.clusterCenters()

In [38]:
import pandas as pd
import numpy as np
import math

In [39]:
# Define columns of the Pandas dataframe
column_list = ['cluster ID', 'size'] #mirai ratio
for feature in input_features:
    column_list.append(feature)
clusters_summary_df_day1 = pd.DataFrame( columns = column_list )
clusters_summary_df_day2 = pd.DataFrame( columns = column_list )

#day1
for i in range(0, total_clusters):
    cluster_i = predictions_day1.where(col('prediction')==i)
    cluster_i_size = cluster_i.count()
    #cluster_i_mirai_count = cluster_i.where(col('mirai')).count()
    #cluster_i_mirai_ratio = cluster_i_mirai_count/cluster_i_size
    #if cluster_i_mirai_count > 0:
        #print("Cluster ", i, "; Mirai Ratio:", cluster_i_mirai_ratio, "; Cluster Size: ", cluster_i_size)
    cluster_row = [i, cluster_i_size] #cluster_i_mirai_ratio
    for j in range(0, len(input_features)):
        cluster_row.append(centers_day1[i][j])
    clusters_summary_df_day1.loc[i]= cluster_row

In [40]:
#day2
for i in range(0, total_clusters):
    cluster_i = predictions_day1.where(col('prediction')==i)
    cluster_i_size = cluster_i.count()
    #cluster_i_mirai_count = cluster_i.where(col('mirai')).count()
    #cluster_i_mirai_ratio = cluster_i_mirai_count/cluster_i_size
    #if cluster_i_mirai_count > 0:
        #print("Cluster ", i, "; Mirai Ratio:", cluster_i_mirai_ratio, "; Cluster Size: ", cluster_i_size)
    cluster_row = [i, cluster_i_size]
    for j in range(0, len(input_features)):
        cluster_row.append(centers_day2[i][j])
    clusters_summary_df_day2.loc[i]= cluster_row

In [41]:
path4= "/storage/home/aka5922/Clusters_Summary_267OHE_k267_D1_1.csv"
path4_2= "/storage/home/aka5922/Clusters_Summary_267OHE_k267_D2_1.csv"
clusters_summary_df_day1.to_csv(path4, header=True)
clusters_summary_df_day2.to_csv(path4_2, header=True)

In [42]:
J_Nearest_Clusters_D2_D1_df = pd.DataFrame(columns=["D2Cindex", "D2Csize", "D1NearestCindex", "D1Csize", "Similarity"])
# Cluster Centers of Day 2
centers_D2 = centers_day1
# Cluster Centers of Day 1
centers_D1 = centers_day2
i = 0
for center2 in centers_D2:
    j = 0
    # Initialize shortest_dist
    highest_similarity = 0
    closest_cluster = -1
    for center1 in centers_D1:
        # skip clusters whose cluster centers are all zeros
        if np.sum(center1) != 0:
            fuzzy_intersection = np.minimum(center1, center2)
            fuzzy_union = np.maximum(center1, center2)
            cluster_i_j_similarity = np.sum(fuzzy_intersection)/np.sum(fuzzy_union)
            if cluster_i_j_similarity > highest_similarity:
                highest_similarity = cluster_i_j_similarity
                closest_cluster = j
        j = j+1
    if closest_cluster != -1:
        J_Nearest_Clusters_D2_D1_df.loc[i]=[ i, summary_day2.clusterSizes[i], closest_cluster, summary_day1.clusterSizes[closest_cluster], highest_similarity]
    else:
        J_Nearest_Clusters_D2_D1_df.loc[i]=[ i, summary_day2.clusterSizes[i], -1, 0, 0 ]
    i = i+1

In [43]:
nearest_clusters_path="/storage/home/aka5922/J_nearest_clusters_D1_D2_union_1.csv"
J_Nearest_Clusters_D2_D1_df.to_csv(nearest_clusters_path, header=True)